In [21]:
# Augment images
import imgaug.augmenters as iaa

############################################################
# Define augmentation pipeline
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontally flip 50% of images
    iaa.Crop(percent=(0, 0.1)), # crop images by 0-10% of their height/width
    iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.5))), # apply Gaussian blur with sigma between 0 and 0.5
    iaa.Affine(rotate=(-45, 45)) # rotate images by -45 to 45 degrees
], random_order=True)  # apply augmenters in random order

#############################################################

path = os.getcwd()
path = path + "\\train_top"

jpg_files = [f for f in os.listdir(path) if f.endswith('.jpg')]

print(jpg_files)
for image in jpg_files:
    img = cv2.imread(path + "\\" + image)
    augmented_image = seq.augment_image(img)
    cv2.imwrite(image + "_augment.jpg", augmented_image)

['20240316_278_640_480_2.jpg', '20240317_279_640_480_2.jpg', '20240317_279_800_600_2.jpg', '20240318_279_640_480_2 (1).jpg', '20240318_279_640_480_2.jpg', '20240318_279_800_600_2 (1).jpg', '20240318_279_800_600_2.jpg', '20240319_279_640_480_2.jpg', '20240319_279_800_600_2 (1).jpg', '20240319_279_800_600_2.jpg', '20240320_282_640_480_2(1).jpg', '20240320_282_640_480_2(2).jpg', '20240320_282_640_480_2.jpg', '20240321_285_640_480_2 (1).jpg', '20240321_285_640_480_2.jpg', '20240321_285_800_600_2.jpg', '20240322_285_640_480_2.jpg', '20240322_285_800_600_2(1).jpg', '20240322_285_800_600_2.jpg', '20240323_286_640_480_2(1).jpg', '20240323_286_640_480_2.jpg', '20240323_286_800_600_2(1).jpg', '20240323_286_800_600_2.jpg', '20240324_287_640_480_2(1).jpg', '20240324_287_640_480_2.jpg', '20240324_287_800_600_2(1).jpg', '20240324_287_800_600_2.jpg', '20240325_289_640_480_2(1).jpg', '20240325_289_640_480_2.jpg', '20240325_289_800_600_2.jpg', '20240326_289_640_480_2(1).jpg', '20240326_289_640_480_2.jp

In [3]:
# Train model
import os

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from joblib import dump

X=[]
y=[]
initial_weight = 277

path = os.getcwd()
path = path + "\\train"

jpg_files = [f for f in os.listdir(path) if f.endswith('.jpg')]

print(jpg_files)
for image in jpg_files:
    features = [0]
    percentagegreen, leafsize = percentage_green__leafsize(path + "\\" + image)
    features.append(percentagegreen)
    features.append(leafsize)
    features.append(plant_health__leaf_number(path + "\\" + image))

    X.append(features)
    y.append(int(image[9:image.index("_", 9)]))
    
y= [num-initial_weight for num in y]
print(X)
print(y)
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


#######################################
# SVR
#######################################
# SVR Model
# Define the parameter grid
svr_param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf', 'linear', 'poly']}

# Create the GridSearchCV object
svr_grid_search = GridSearchCV(SVR(), svr_param_grid, cv=5)

# Fitting the model
svr_grid_search.fit(X_train, y_train)
# Print the best parameters found
print("Best SVR parameters found: ", svr_grid_search.best_params_)

svr = svr_grid_search.best_estimator_
# Make predictions using the testing set
svr_y_pred = svr.predict(X_test)

print("SVR results:")
svr_rmse = np.sqrt(mean_squared_error(y_test, svr_y_pred))
print('Root Mean Squared Error:', svr_rmse)

#######################################
# DTR
#######################################
# Decision Tree Regressor Model
# Define the parameter grid
dtr_param_grid = {
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Create the GridSearchCV object
dtr_grid_search = GridSearchCV(DecisionTreeRegressor(random_state=42), dtr_param_grid, cv=5, scoring='neg_mean_squared_error')

# Fitting the model
dtr_grid_search.fit(X_train, y_train)
# Print the best parameters found
print("Best DTR parameters found: ", dtr_grid_search.best_params_)

dtr = dtr_grid_search.best_estimator_
# Make predictions using the testing set
dt_y_pred = dtr.predict(X_test)

print("DTR results:")
dtr_rmse = np.sqrt(mean_squared_error(y_test, dt_y_pred))
print('Root Mean Squared Error:', dtr_rmse)

#######################################
# XGBoost
#######################################
# XGBoost regressor model
# Define the parameter grid
xgb_param_grid = {
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.5, 0.7, 0.9],
    'colsample_bytree': [0.5, 0.7, 0.9]
}

# Create the XGBRegressor object
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Create the GridSearchCV object
xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=xgb_param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)

# xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.05,
#                 max_depth = 5, alpha = 10, n_estimators = 20)

# Fitting the model
xgb_grid_search.fit(X_train, y_train)
# Print the best parameters found
print("Best XGB parameters found: ", xgb_grid_search.best_params_)

xgb_opt = xgb_grid_search.best_estimator_
# Make predictions using the testing set
xgb_y_pred = xgb_opt.predict(X_test)

print("XGB results:")
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_y_pred))
print("Root Mean Squared Error:", xgb_rmse)


# Saving the trained model
svr_model_filename = 'svr_yield.joblib'
dtr_model_filename = 'dtr_yield.joblib'
xgb_model_filename = 'xgb_yield.joblib'
dump(svr, svr_model_filename)
dump(dtr, dtr_model_filename)
dump(xgb_opt, xgb_model_filename)

total_rmse = (1/svr_rmse) + (1/dtr_rmse) + (1/xgb_rmse)
svr_weighted_pred = (1/svr_rmse) / total_rmse
dtr_weighted_pred = (1/dtr_rmse) / total_rmse
xgb_weighted_pred = (1/xgb_rmse) / total_rmse

print(total_rmse)
print(svr_weighted_pred)
print(dtr_weighted_pred)
print(xgb_weighted_pred)

['20240316_278_1137_640.jpg', '20240316_278_320_240.jpg', '20240316_278_640_480 (1).jpg', '20240316_278_640_480.jpg', '20240316_278_640_480_2.jpg', '20240316_278_800_600.jpg', '20240317_279_640_480.jpg', '20240317_279_640_480_2.jpg', '20240317_279_800_600.jpg', '20240317_279_800_600_2.jpg', '20240318_279_640_480_ (1).jpg', '20240318_279_640_480_.jpg', '20240318_279_640_480_2 (1).jpg', '20240318_279_640_480_2.jpg', '20240318_279_800_600_.jpg', '20240318_279_800_600_2 (1).jpg', '20240318_279_800_600_2.jpg', '20240319_279_640_480_.jpg', '20240319_279_640_480_2.jpg', '20240319_279_800_600_.jpg', '20240319_279_800_600_2 (1).jpg', '20240319_279_800_600_2.jpg', '20240320_282_640_480_(1).jpg', '20240320_282_640_480_.jpg', '20240320_282_640_480_2(1).jpg', '20240320_282_640_480_2(2).jpg', '20240320_282_640_480_2.jpg', '20240320_282_800_600_.jpg', '20240321_285_640_480_.jpg', '20240321_285_640_480_2 (1).jpg', '20240321_285_640_480_2.jpg', '20240321_285_800_600_.jpg', '20240321_285_800_600_2.jpg',

image 1/1 C:\Users\Qi Ren\Desktop\FYP\train\20240318_279_640_480_2 (1).jpg: 384x480 4 Leafs, 127.1ms
Speed: 11.8ms preprocess, 127.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 480)
Results saved to runs\detect\predict21
4.0
0.002625
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 5.414
Area: 0.500
Length: 17.071
Area: 0.500
Length: 5.657
Area: 2.000
Length: 7.414
Area: 3.500
Length: 27.556
Area: 40.500
Length: 5.657
Area: 2.000
Length: 39.556
Area: 47.500
Length: 23.314
Area: 38.000
Length: 25.071
Area: 39.500
16.363636363636363
---------

image 1/1 C:\Users\Qi Ren\Desktop\FYP\train\20240318_279_640_480_2.jpg: 384x480 4 Leafs, 146.2ms
Speed: 0.0ms preprocess, 146.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 480)
Results saved to runs\detect\predict21
4.0
0.003775
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 26.728
Area: 39.500
Length: 38.728
Area: 44.500
Length: 5.657
Area: 2.000
Length: 23.899
Area: 37.500
Length: 26.728

image 1/1 C:\Users\Qi Ren\Desktop\FYP\train\20240321_285_640_480_.jpg: 384x480 5 Leafs, 135.4ms
Speed: 1.0ms preprocess, 135.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 480)
Results saved to runs\detect\predict21
5.0
0.00335
Length: 32.728
Area: 1.500
Length: 19.071
Area: 0.500
Length: 13.899
Area: 10.500
Length: 16.243
Area: 12.500
Length: 32.971
Area: 62.000
Length: 33.556
Area: 64.500
Length: 8.828
Area: 1.000
Length: 40.385
Area: 1.500
19.25
---------

image 1/1 C:\Users\Qi Ren\Desktop\FYP\train\20240321_285_640_480_2 (1).jpg: 384x480 5 Leafs, 131.1ms
Speed: 3.3ms preprocess, 131.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 480)
Results saved to runs\detect\predict21
5.0
0.003325
Length: 47.799
Area: 2.000
Length: 22.828
Area: 1.000
Length: 5.657
Area: 2.000
Length: 6.828
Area: 3.000
Length: 31.556
Area: 55.500
Length: 33.314
Area: 59.000
Length: 8.828
Area: 1.000
Length: 11.657
Area: 1.000
Length: 36.385
Area: 3.500
Length: 5.657
Area: 2.000


Results saved to runs\detect\predict21
5.0
0.006975
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 9.657
Area: 6.000
Length: 43.799
Area: 1.000
Length: 5.657
Area: 2.000
Length: 23.314
Area: 1.000
Length: 8.000
Area: 4.000
Length: 46.627
Area: 125.000
Length: 62.042
Area: 134.500
Length: 59.113
Area: 3.000
Length: 5.657
Area: 2.000
Length: 26.728
Area: 2.500
Length: 12.485
Area: 10.000
Length: 32.971
Area: 15.000
22.285714285714285
---------

image 1/1 C:\Users\Qi Ren\Desktop\FYP\train\20240323_286_640_480_2.jpg: 384x480 5 Leafs, 142.6ms
Speed: 4.3ms preprocess, 142.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 480)
Results saved to runs\detect\predict21
5.0
0.003775
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 9.071
Area: 0.500
Length: 15.314
Area: 12.000
Length: 17.657
Area: 14.000
Length: 81.841
Area: 3.500
Length: 10.828
Area: 1.000
Length: 8.243
Area: 0.500
Length: 17.899
Area: 22.500

image 1/1 C:\Users\Qi Ren\Desktop\FYP\train\20240325_289_640_480_(1).jpg: 384x480 6 Leafs, 134.9ms
Speed: 6.0ms preprocess, 134.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 480)
Results saved to runs\detect\predict21
6.0
0.0033
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 10.485
Area: 7.000
Length: 24.728
Area: 8.500
Length: 33.799
Area: 2.000
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 6.828
Area: 1.000
Length: 18.728
Area: 24.500
Length: 29.799
Area: 25.000
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 23.314
Area: 30.000
Length: 25.071
Area: 31.500
9.59375
---------

image 1/1 C:\Users\Qi Ren\Desktop\FYP\train\20240325_289_640_480_.jpg: 384x480 6 Leafs, 130.5ms
Speed: 2.0ms preprocess, 130.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 480)
Results saved to runs\detect\predict21
6.0
0.01
Length: 7.414
Area: 0.500
Length: 14.485
Area: 1.000
Length: 36.3

image 1/1 C:\Users\Qi Ren\Desktop\FYP\train\20240326_289_800_600_.jpg: 384x480 7 Leafs, 134.5ms
Speed: 2.0ms preprocess, 134.5ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 480)
Results saved to runs\detect\predict21
7.0
0.009025
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 73.355
Area: 1.500
Length: 8.243
Area: 0.500
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 46.042
Area: 118.500
Length: 46.627
Area: 124.000
Length: 7.657
Area: 4.000
Length: 8.828
Area: 5.000
Length: 45.456
Area: 58.000
Length: 32.142
Area: 62.000
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 3.414
Area: 0.500
Length: 32.142
Area: 1.000
Length: 15.314
Area: 15.000
Length: 45.213
Area: 17.500
Length: 24.142
Area: 4.000
22.605263157894736
---------

image 1/1 C:\Users\Qi Ren\Desktop\FYP\train\20240326_289_800_600_2(1).jpg: 384x480 7 Leafs, 126.4ms
Speed: 1.0ms preprocess, 126.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 480)
Results saved t

image 1/1 C:\Users\Qi Ren\Desktop\FYP\train\20240327_291_800_600_2.jpg: 384x480 10 Leafs, 115.8ms
Speed: 2.0ms preprocess, 115.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 480)
Results saved to runs\detect\predict21
10.0
0.010225
Length: 21.899
Area: 25.500
Length: 23.071
Area: 26.500
Length: 5.414
Area: 0.500
Length: 100.912
Area: 7.000
Length: 12.485
Area: 10.000
Length: 14.243
Area: 11.500
Length: 35.799
Area: 1.000
Length: 8.828
Area: 1.000
Length: 27.899
Area: 0.500
Length: 5.657
Area: 2.000
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 8.000
Area: 4.000
Length: 23.556
Area: 34.500
Length: 25.314
Area: 38.000
Length: 49.213
Area: 147.500
Length: 49.799
Area: 152.000
27.5
---------

image 1/1 C:\Users\Qi Ren\Desktop\FYP\train\20240328_293_640_480_.jpg: 384x480 7 Leafs, 121.3ms
Speed: 2.0ms preprocess, 121.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 480)
Results saved to runs\detect\predict21
7.0
0.01815
Length: 7.657
Area: 4.000


Results saved to runs\detect\predict21
7.0
0.020125
Length: 24.728
Area: 1.500
Length: 20.485
Area: 1.000
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 5.657
Area: 2.000
Length: 7.414
Area: 3.500
Length: 5.657
Area: 2.000
Length: 49.314
Area: 1.000
Length: 68.527
Area: 252.500
Length: 8.000
Area: 4.000
Length: 47.456
Area: 6.000
Length: 68.527
Area: 259.500
Length: 7.414
Area: 0.500
Length: 5.657
Area: 2.000
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 8.000
Area: 4.000
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 5.657
Area: 2.000
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 8.000
Area: 4.000
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 13.071
Area: 0.500
Length: 10.828
Area: 1.000
Length: 11.071
Area: 0.500
Length: 60.284
Area: 222.000
Length: 109.841
Area: 6.500
Length: 62.042
Area: 228.500
31.53030303030303
---------

image 1/1 C:\Users\Qi Ren\Desktop\FYP\tr

image 1/1 C:\Users\Qi Ren\Desktop\FYP\train\20240330_297_800_600_.jpg: 384x480 7 Leafs, 112.1ms
Speed: 0.0ms preprocess, 112.1ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 480)
Results saved to runs\detect\predict21
7.0
0.015825
Length: 45.456
Area: 135.000
Length: 47.213
Area: 144.500
Length: 20.728
Area: 0.500
Length: 6.828
Area: 1.000
Length: 46.284
Area: 105.000
Length: 5.657
Area: 2.000
Length: 48.042
Area: 109.500
Length: 31.556
Area: 0.500
Length: 26.728
Area: 5.500
Length: 6.828
Area: 1.000
Length: 5.657
Area: 2.000
Length: 48.042
Area: 3.500
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 8.828
Area: 1.000
Length: 6.828
Area: 1.000
Length: 148.468
Area: 6.500
30.852941176470587
---------

image 1/1 C:\Users\Qi Ren\Desktop\FYP\train\20240330_297_800_600_2(1).jpg: 384x480 6 Leafs, 142.3ms
Speed: 0.0ms preprocess, 142.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 480)
Results saved to runs\detect\predict21
6.0
0.0163
Length: 10.828


Best SVR parameters found:  {'C': 100, 'gamma': 1, 'kernel': 'poly'}
SVR results:
Root Mean Squared Error: 3.691471742543888
Best DTR parameters found:  {'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5}
DTR results:
Root Mean Squared Error: 3.525495268143599
Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best XGB parameters found:  {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 3, 'subsample': 0.9}
XGB results:
Root Mean Squared Error: 2.4623775420555605
0.9606543210931906
0.28198974258125753
0.2952655122904386
0.42274474512830384


In [4]:
# Main function in Cloud
# Get new image POST by ESP32-CAM
# Read the images
from joblib import load

path = "C:\\Users\\Qi Ren\\Desktop\\FYP\\test\\"
img = "20240320_282_800_600_2.jpg"
new_X = [0]

# Process the image to get desired features

percentagegreen_new, leafsize_new = percentage_green__leafsize(path+img)
new_X.append(percentagegreen_new)
new_X.append(leafsize_new)
new_X.append(plant_health__leaf_number(path+img))

new_X = np.array(new_X).reshape(1, -1)

model_filename = 'plant_yield_model.joblib'
svr_model = load(model_filename)


svr_estimated_yield = svr_model.predict(new_X)
print("SVR Prediction: ", svr_estimated_yield)
dtr_estimated_yield = dtr.predict(new_X)
print("DTR Prediction: ", dtr_estimated_yield)
xgb_estimated_yield = xgb_opt.predict(new_X)
print("XGB Prediction: ", xgb_estimated_yield)

ensemble_yield = svr_estimated_yield*svr_weighted_pred + dtr_estimated_yield*dtr_weighted_pred + xgb_estimated_yield*xgb_weighted_pred
print("")
print("Ensemble pred: ", ensemble_yield)
print("Actual yield: ", int(img[9:img.index("_", 9)])- initial_weight)

0.0066
Length: 17.899
Area: 21.500
Length: 19.071
Area: 23.500
Length: 27.314
Area: 1.000
Length: 26.142
Area: 44.000
Length: 27.314
Area: 47.000
Length: 44.042
Area: 105.500
Length: 45.213
Area: 109.500
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
Length: 59.113
Area: 2.000
Length: 5.657
Area: 2.000
Length: 8.000
Area: 4.000
30.5
---------

image 1/1 C:\Users\Qi Ren\Desktop\FYP\test\20240320_282_800_600_2.jpg: 384x480 4 Leafs, 133.1ms
Speed: 1.0ms preprocess, 133.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 480)
Results saved to runs\detect\predict21
4.0
SVR Prediction:  [     7.3195]
DTR Prediction:  [          4]
XGB Prediction:  [      2.393]

Ensemble pred:  [     4.2567]
Actual yield:  5


In [1]:
import cv2 
import numpy as np

def percentage_green__leafsize(img):
    #######################################################
    # Percentage green
    #######################################################
    
    # Using the image
    image = cv2.imread(img)
    # Resizing the image
    image = cv2.resize(image, (200, 200)) 

    res = image.copy()
    
    # Convert Image to Image HSV 
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) 

    # Defining lower and upper bound HSV values
    lower = np.array([35, 65, 120]) 
    upper = np.array([50, 250, 250]) 

    # Defining mask for detecting color 
    mask = cv2.inRange(hsv, lower, upper)

    # Get percentage of green in image
    percentage_green = np.count_nonzero(mask)/40000
    print(percentage_green)

    
    ###################################################
    # For debugging
    ###################################################
    
#     # Get CSV file for analysis
#     np.savetxt("tst.csv",
#             mask,
#             delimiter =", ",
#             fmt ='% s')
    
#     # Get image output and what is identified as green
#     res = cv2.bitwise_and(image,image,mask = mask)
#     cv2.imwrite(img[:-3] + "_mask.jpg", res)


    #######################################################
    # Leaf Size
    #######################################################
    
    # convert to gray 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # threshold the gray
    th = 100 
    th, threshed = cv2.threshold(gray, th, 255,  cv2.THRESH_BINARY)
    
    # Get identified green as grey image
    img_search = cv2.bitwise_and(mask, threshed)
    
    # Canny Edge Detection
    edges = cv2.Canny(image=img_search, threshold1=80, threshold2=300) 
    
    ###################################################
    # For debugging
    ###################################################
#     cv2.imwrite("edge_{}.png".format(1), edges)

    cnts = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2]

    counts = 0
    total_area = 0

    for cnt in cnts:
        arclen = cv2.arcLength(cnt, True)
        area = cv2.contourArea(cnt)
        if area > 0:
            total_area += area
            cv2.drawContours(res, [cnt], -1, (0,255,0), 3, cv2.LINE_AA)
            print("Length: {:.3f}\nArea: {:.3f}".format(arclen, area))

#             cv2.imwrite("edge_check_{}.png".format(counts), res)
            counts +=1

    leaf_size = total_area/counts
    print(leaf_size)
    leaf_size = leaf_size/ (leaf_size + 1)
    print("---------")
    


    
    return percentage_green, leaf_size

In [70]:
# Check range of values to be classified as green

green = np.uint8([[[112, 181, 148]]]) 
  
# Convert Green color to Green HSV 
hsv_green = cv2.cvtColor(green, cv2.COLOR_BGR2HSV) 
# Print HSV Value for Green color 
print(hsv_green) 

#-------------------------------------------------------
check = np.uint8([[[73, 141, 106]]]) 
# Convert Green color to Green HSV 
hsv_check = cv2.cvtColor(check, cv2.COLOR_BGR2HSV) 
# Print HSV Value for Green color 
print(hsv_check) 


check2 = np.uint8([[[45, 146, 108]]]) 
# Convert Green color to Green HSV 
hsv_check2 = cv2.cvtColor(check2, cv2.COLOR_BGR2HSV) 
# Print HSV Value for Green color 
print(hsv_check2) 


check3 = np.uint8([[[47, 151, 120]]]) 
# Convert Green color to Green HSV 
hsv_check3 = cv2.cvtColor(check3, cv2.COLOR_BGR2HSV) 
# Print HSV Value for Green color 
print(hsv_check3) 

check4 = np.uint8([[[12, 122, 71]]]) 
# Convert Green color to Green HSV 
hsv_check4 = cv2.cvtColor(check4, cv2.COLOR_BGR2HSV) 
# Print HSV Value for Green color 
print(hsv_check4) 

check5 = np.uint8([[[9, 113, 72]]]) 
# Convert Green color to Green HSV 
hsv_check5 = cv2.cvtColor(check5, cv2.COLOR_BGR2HSV) 
# Print HSV Value for Green color 
print(hsv_check5) 

[[[ 44  97 181]]]
[[[ 45 123 141]]]
[[[ 41 176 146]]]
[[[ 39 176 151]]]
[[[ 44 230 122]]]
[[[ 42 235 113]]]


In [9]:
image = cv2.imread("C:\\Users\\Qi Ren\\Desktop\\FYP\\test\\20240325_289_800_600_2(1).jpg")
# Resizing the image
image = cv2.resize(image, (200, 200)) 

res = image.copy()
# Convert Image to Image HSV 
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) 

# Defining lower and upper bound HSV values
lower = np.array([35, 65, 120]) 
upper = np.array([50, 250, 250])

# Defining mask for detecting color 
mask = cv2.inRange(hsv, lower, upper)

# Get percentage of green in image
percentage_green = np.count_nonzero(mask)/40000
print(percentage_green)





# Get image output and what is identified as green
res = cv2.bitwise_and(image,image,mask = mask)
cv2.imwrite("TESTTTT" + "_mask.jpg", res)


cnts = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2]



# convert to gray 
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# threshold the gray
th = 100 
th, threshed = cv2.threshold(gray, th, 255,  cv2.THRESH_BINARY)

# Get identified green as grey image
img_search = cv2.bitwise_and(mask, threshed)

# Canny Edge Detection
edges = cv2.Canny(image=img_search, threshold1=50, threshold2=300) 


###################################################
# For debugging
###################################################
cv2.imwrite("edge_{}.png".format("000"), edges)




## calcualte 
for cnt in cnts:
    arclen = cv2.arcLength(cnt, True)
    area = cv2.contourArea(cnt)
    cv2.drawContours(res, [cnt], -1, (0,255,0), 3, cv2.LINE_AA)
    print("Length: {:.3f}\nArea: {:.3f}".format(arclen, area))

cv2.imwrite("check_{}.png".format(2), res)

0.01265
Length: 0.000
Area: 0.000
Length: 0.000
Area: 0.000
Length: 3.414
Area: 0.500
Length: 4.000
Area: 0.000
Length: 5.657
Area: 2.000
Length: 5.657
Area: 2.000
Length: 9.657
Area: 6.000
Length: 73.941
Area: 162.000
Length: 5.657
Area: 2.000
Length: 0.000
Area: 0.000
Length: 42.728
Area: 97.500
Length: 34.142
Area: 57.000
Length: 0.000
Area: 0.000
Length: 27.314
Area: 22.000
Length: 7.657
Area: 4.000
Length: 12.485
Area: 10.000
Length: 42.385
Area: 69.500


True

In [2]:
from ultralytics import YOLO
import tensorflow as tf

#Load a pretrained YOLOv8n model
yolo_model = YOLO('27MarModel.pt')

# img = "C:\\Users\\Qi Ren\\Desktop\\FYP\\test\\20240320_282_800_600_2.jpg"

def plant_health__leaf_number(img):
    #Run inference on the source
    results = yolo_model(source=img, show=False, conf=0.5, save=True) #generator of results

    num_leaves = 0
    for result in results:
        num_leaves = tf.math.count_nonzero(result.boxes.cls)
    
    num_leaves = float(num_leaves)
    print(num_leaves)
    num_leaves = num_leaves / (num_leaves + 1)
    
    return num_leaves